In [ ]:
__version__ = '1.0.0-alpha.1'
__author__ = 'Martino Pulici'

In [ ]:
import pandas as pd
import pulp

In [ ]:
from branch_and_bound import branch_and_bound

In [ ]:
prob_mon = pulp.LpProblem("Monday", pulp.LpMinimize)
prob_tue = pulp.LpProblem("Tuesday", pulp.LpMinimize)
prob_wed = pulp.LpProblem("Wednesday", pulp.LpMinimize)
probs = [prob_mon,prob_tue,prob_wed]

In [ ]:
import csv
df = pd.read_csv("food.csv")
with open('dish.csv') as f:
    reader = csv.reader(f)
    dish_list = list(reader)

In [ ]:
nutrient_limits = {'Energy': 2000,
          'Fat': 70,
          'Saturates': 20,
          'Carbohydrates': 260,
          'Sugars': 90,
          'Protein': 50
         }

In [ ]:
class dish:
    def __init__(self, name):
        self.name = name
        self.cost = 0
        self.nutrients = {nut:0 for nut in nutrient_limits.keys()}

In [ ]:
labels = list(df['Food'])
costs = dict(zip(labels,df['Cost']))

In [ ]:
nutrients = {}
for i in range(len(labels)):
    nutrients[labels[i]] = {}
    for nut in nutrient_limits.keys():
        nutrients[labels[i]][nut] = df[nut][i]*10

In [ ]:
dish_labels = []
dishes = []
for i in range(len(dish_list)):
    d = dish_list[i]
    dishes.append(dish(d[0]))
    dish_labels.append(d[0])
    for j in range(1,len(d),2):
        ingredient = d[j]
        quantity = float(d[j+1])/1000
        dishes[i].cost += costs[ingredient]*quantity
        for nut in dishes[i].nutrients.keys():
            dishes[i].nutrients[nut] += nutrients[ingredient][nut]*quantity

In [ ]:
food_mon = pulp.LpVariable.dicts("Food_Mon",dish_labels,0)
food_tue = pulp.LpVariable.dicts("Food_Tue",dish_labels,0)
food_wed = pulp.LpVariable.dicts("Food_Wed",dish_labels,0)
foods = [food_mon,food_tue,food_wed]

In [ ]:
for nut in nutrient_limits.keys():
    for i in range(len(probs)):
        probs[i] += pulp.lpSum([dish.cost*foods[i][dish.name] for dish in dishes])
        probs[i] += pulp.lpSum([dish.nutrients[nut] * foods[i][dish.name] for dish in dishes]) >= nutrient_limits[nut]*0.8
        probs[i] += pulp.lpSum([dish.nutrients[nut] * foods[i][dish.name] for dish in dishes]) <= nutrient_limits[nut]*1.2

In [ ]:
new = []

new.append(branch_and_bound(prob_mon))
new[0].solve()

for i in range(1,len(probs)):
    for j in range(0,i):
        for k in range(len(new[j].variables())):
            if new[j].variables()[k].varValue:
                probs[i] += probs[i].variables()[k] == 0
    new.append(branch_and_bound(probs[i]))
    new[-1].solve()

In [ ]:
print('Cost =', pulp.value(sum([n.objective for n in new])), '€')

In [ ]:
for day in ["_Mon_","_Tue_","_Wed_"]:
    print()
    print(day[1:4].upper())
    for n in new:
        for v in n.variables():
            if v.varValue and day in v.name:
                print(v.name[9:], '=', '{0:.0f}'.format(v.varValue))